<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Rafael De La Sotta
- Nombre de alumno 2: Andrés Medina


### **Link de repositorio de GitHub:** `https://github.com/rafadls/LPC/tree/main/L8`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# ignorar warnings
import warnings
warnings.filterwarnings("ignore")

# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# GridSearch
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

# Metricas de evaluación
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

Requirement already up-to-date: plotly in c:\users\rafael\anaconda3\lib\site-packages (5.4.0)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

> En el siguiente código se realiza la normalización de los atributos de interes y la vectorizaión de la columna 'history_text'. Todo esto se realiza de forma paralela dentro de un ColumnTransformer.

In [15]:
preprocessing = ColumnTransformer([
    ('Scale', MinMaxScaler(), [
        'intelligence_score',
        'strength_score',
        'speed_score', 
        'durability_score', 
        'power_score', 
        'combat_score',]), 
    ('Vectorizer', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), decode_error='ignore'),'history_text')])

In [16]:
preprocessing.fit(df_comics)

ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                 ['intelligence_score', 'strength_score',
                                  'speed_score', 'durability_score',
                                  'power_score', 'combat_score']),
                                ('Vectorizer',
                                 CountVectorizer(decode_error='ignore',
                                                 ngram_range=(1, 2),
                                                 tokenizer=<__main__.StemmerTokenizer object at 0x000001AAF27B0520>),
                                 'history_text')])

In [17]:
Vectorized = preprocessing.transform(df_comics)

In [23]:
Vectorized.shape

(1285, 307713)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

> Se utiliza el ColumnTransformer de vecotrización en conjunto con SelectPercentile y MultinomialNB(). Estos tres procesos son incluidos en un mismo pipeline. 

> Hecho el pipeline se separan los datos en entrenamiento y prueba, los cuales son utilizados según su respectiva función. 

> Finalmente se realiza un classification_report

In [24]:
preprocessing = ColumnTransformer([
    ('Scale', MinMaxScaler(), [
        'intelligence_score',
        'strength_score',
        'speed_score', 
        'durability_score', 
        'power_score', 
        'combat_score',]), 
    ('Vectorizer', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), decode_error='ignore'),'history_text')])

In [25]:
pipeline_clasificador = Pipeline([('preprocessing', preprocessing),
                                  ('SelectPercentiles', SelectPercentile(f_classif, percentile=90)),
                                  ('multinomialnb', MultinomialNB())])

In [26]:
features = df_comics.drop(columns=['alignment'])
labels = df_comics.loc[:, 'alignment']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, shuffle=True, stratify=labels)

In [27]:
pipeline_clasificador.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('Vectorizer',
                                                  CountVectorizer(decode_error='ignore',
                                                                  ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x000001AAF6B5E6A0>),
                        

In [28]:
y_pred = pipeline_clasificador.predict(X_test)

In [29]:
print('Matriz de confusión: \n')
print(confusion_matrix(y_test, y_pred, labels=pipeline_clasificador.classes_))

Matriz de confusión: 

[[ 10 132   0]
 [  6 240   0]
 [  1  36   0]]


In [30]:
print('\nReporte de Clasificación: \n')
print(classification_report(y_test, y_pred, target_names=pipeline_clasificador.classes_), )


Reporte de Clasificación: 

              precision    recall  f1-score   support

         Bad       0.59      0.07      0.13       142
        Good       0.59      0.98      0.73       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.59       425
   macro avg       0.39      0.35      0.29       425
weighted avg       0.54      0.59      0.47       425



* **¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.**

> No se recomienda, esto debido a que no funciona de una forma confiable.

> El modelo tiene un rendimiento bajo pero equilibrado en el **precision**. El modelo tiene una seguridad similar para decir cada clase. Por otro lado, el modelo tiene un rendimiento desequilibrado en el **recall**. El modelo tiende a decir que la clase es Good por sobre Bad. Ambas métricas anteriormente analizadas tienen su efecto en el f1-score.

> En parte los malos resultados se pueden deber a que no se balancean los conjuntos. 

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

> A continuación se utiliza **HalvingGridSearchCV** con el fin de encontrar los mejores parámetros y clasificadores enfocados en este problema en particular 

> Los resultados alcanzados con el mejor modelo son mejores que los alcanzados en la sección anterior. Además de tener mejores metricas tienen sus estadisticas "equitlibradas" entre las clases. Como consecuencia de esto, el modelo tiene una mayor confiabiliad.


In [47]:
preprocessing = ColumnTransformer([
    ('Scale', MinMaxScaler(), [
        'intelligence_score',
        'strength_score',
        'speed_score', 
        'durability_score', 
        'power_score', 
        'combat_score',]), 
    ('Vectorizer', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2), decode_error='ignore'),'history_text')])

pipeline_clasificador = Pipeline([('preprocessing', preprocessing),
                                  ('Selection', SelectPercentile(f_classif, percentile=90)),
                                  ('clf', MultinomialNB())])

In [58]:
params = [
        {'preprocessing__Vectorizer__ngram_range': [(1,1), (1,2), (1,3)]},
        {'Selection__percentile': [20, 40, 60, 80]},
       # LogisticRegression
       {'clf': [LogisticRegression()],
        'clf__penalty': ['l2','none'],
        'clf__C': [0.1, 1, 10, 100]},
       # RandomForestClassifier    
       {'clf': [RandomForestClassifier()],
        'clf__n_estimators': [50,100,200],
        'clf__criterion': ['gini', 'entropy']},
       ]

hgs = HalvingGridSearchCV(pipeline_clasificador, params,verbose=10)

In [59]:
hgs.fit(X_train, y_train)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 95
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 21
n_resources: 95
Fitting 5 folds for each of 21 candidates, totalling 105 fits
[CV 1/5; 1/21] START preprocessing__Vectorizer__ngram_range=(1, 1)..............
[CV 1/5; 1/21] END preprocessing__Vectorizer__ngram_range=(1, 1);, score=(train=1.000, test=0.579) total time=   1.5s
[CV 2/5; 1/21] START preprocessing__Vectorizer__ngram_range=(1, 1)..............
[CV 2/5; 1/21] END preprocessing__Vectorizer__ngram_range=(1, 1);, score=(train=0.974, test=0.474) total time=   1.7s
[CV 3/5; 1/21] START preprocessing__Vectorizer__ngram_range=(1, 1)..............
[CV 3/5; 1/21] END preprocessing__Vectorizer__ngram_range=(1, 1);, score=(train=0.921, test=0.684) total time=   1.8s
[CV 4/5; 1/21] START preprocessing__Vectorizer__ngram_range=(1, 1)..............
[CV 4/5; 1/21] END preprocessing__Vectorizer__ngram_range=(1, 1

[CV 3/5; 9/21] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=none;, score=(train=0.987, test=0.421) total time=   2.5s
[CV 4/5; 9/21] START clf=LogisticRegression(), clf__C=0.1, clf__penalty=none....
[CV 4/5; 9/21] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=none;, score=(train=1.000, test=0.579) total time=   1.6s
[CV 5/5; 9/21] START clf=LogisticRegression(), clf__C=0.1, clf__penalty=none....
[CV 5/5; 9/21] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=none;, score=(train=1.000, test=0.632) total time=   2.0s
[CV 1/5; 10/21] START clf=LogisticRegression(), clf__C=1, clf__penalty=l2.......
[CV 1/5; 10/21] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2;, score=(train=1.000, test=0.684) total time=   2.8s
[CV 2/5; 10/21] START clf=LogisticRegression(), clf__C=1, clf__penalty=l2.......
[CV 2/5; 10/21] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2;, score=(train=1.000, test=0.474) total time=   3.2s
[CV 3/5; 10/21] START clf=LogisticRegressi

[CV 2/5; 17/21] END clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100;, score=(train=1.000, test=0.474) total time=   2.0s
[CV 3/5; 17/21] START clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100
[CV 3/5; 17/21] END clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100;, score=(train=0.987, test=0.684) total time=   2.2s
[CV 4/5; 17/21] START clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100
[CV 4/5; 17/21] END clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100;, score=(train=1.000, test=0.632) total time=   1.3s
[CV 5/5; 17/21] START clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100
[CV 5/5; 17/21] END clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=100;, score=(train=1.000, test=0.526) total time=   1.8s
[CV 1/5; 18/21] START clf=RandomForestClassifier(), clf__criterion=gini, clf__n_estimators=200
[CV 1/5; 18/21] END clf=RandomForest

[CV 1/5; 3/7] END clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50;, score=(train=1.000, test=0.518) total time=   5.0s
[CV 2/5; 3/7] START clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50
[CV 2/5; 3/7] END clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50;, score=(train=1.000, test=0.571) total time=   5.0s
[CV 3/5; 3/7] START clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50
[CV 3/5; 3/7] END clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50;, score=(train=1.000, test=0.607) total time=   6.3s
[CV 4/5; 3/7] START clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50
[CV 4/5; 3/7] END clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50;, score=(train=1.000, test=0.500) total time=   6.1s
[CV 5/5; 3/7] START clf=RandomForestClassifier(), clf__criterion=entropy, clf__n_estimators=50
[CV 5/5; 3/7] END clf=RandomForestCl

[CV 2/5; 3/3] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2;, score=(train=0.996, test=0.661) total time=  24.6s
[CV 3/5; 3/3] START clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2.......
[CV 3/5; 3/3] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2;, score=(train=0.996, test=0.673) total time=  25.1s
[CV 4/5; 3/3] START clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2.......
[CV 4/5; 3/3] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2;, score=(train=0.993, test=0.655) total time=  25.5s
[CV 5/5; 3/3] START clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2.......
[CV 5/5; 3/3] END clf=LogisticRegression(), clf__C=0.1, clf__penalty=l2;, score=(train=0.996, test=0.696) total time=  24.9s


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('Scale',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('Vectorizer',
                                                               

In [60]:
hgs.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('Vectorizer',
                                                  CountVectorizer(decode_error='ignore',
                                                                  ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x000001AAF6967D60>),
                        

In [61]:
y_pred = hgs.predict(X_test)

In [63]:
print('Matriz de confusión: \n')
print(confusion_matrix(y_test, y_pred, labels=hgs.classes_))

Matriz de confusión: 

[[ 76  66   0]
 [ 29 216   1]
 [ 12  25   0]]


In [64]:
print('\nReporte de Clasificación: \n')
print(classification_report(y_test, y_pred, target_names=hgs.classes_), )


Reporte de Clasificación: 

              precision    recall  f1-score   support

         Bad       0.65      0.54      0.59       142
        Good       0.70      0.88      0.78       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.69       425
   macro avg       0.45      0.47      0.46       425
weighted avg       0.62      0.69      0.65       425



## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

> A continuación se evalua el mejor modelo de HalvingGridSearchCV en tres personajes, Vergil, Gorilla Girl y Batcow. Este clasificador nos entrega los siguientes resultados:

* Batcow: **Bad**
* Gorilla Girl: **Good**
* Vergil: **Good**

In [81]:
df_VGB = df_comics_no_label[df_comics_no_label['name'].isin(['Vergil','Gorilla Girl','Batcow'])]
df_VGB.drop_duplicates(inplace=True)
df_VGB = df_VGB.drop(columns=['alignment'])
df_VGB

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [84]:
clases_VGB = hgs.predict(df_VGB)
df_VGB['alignment'] = clases_VGB
df_VGB = df_VGB[['name','alignment']]
df_VGB

,name,alignment
16,Batcow,Bad
40,Gorilla Girl,Good
78,Vergil,Good


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>